## [作業重點]
了解如何使用 Sklearn 中的 hyper-parameter search 找出最佳的超參數

### 作業
請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合

In [1]:
import numpy as np
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
# 讀取酒類資料集
digits = datasets.load_wine()

# 切分訓練集/測試集
x_train, x_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.25, random_state=42)

# 建立模型
reg = GradientBoostingRegressor(random_state=7)

# 先看看使用預設參數得到的結果，約為 0.14 的 MSE
reg.fit(x_train, y_train)
y_pred = reg.predict(x_test)
print(metrics.mean_squared_error(y_test, y_pred))

0.14672894294872418


In [3]:
# Search for good hyperparameter values
# Specify values to grid search over
n_estimators = list(np.arange(10, 60, 20))
max_features = [None, 'sqrt', 'log2'] 
 
hyperparameters = {
    'n_estimators': n_estimators, 
    'max_features': max_features
    }
    
# Grid search using cross-validation
gridCV = GridSearchCV(GradientBoostingRegressor(random_state=7), param_grid=hyperparameters, cv=10, n_jobs=4)

# 開始搜尋最佳參數
grid_result = gridCV.fit(x_train, y_train)

# 印出最佳結果與最佳參數
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best Accuracy: 0.933377 using {'max_features': 'sqrt', 'n_estimators': 50}


D:\Anaconda\envs\python3.6\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [4]:
# 使用最佳參數重新建立模型
reg_bestparam = GradientBoostingRegressor(max_features=grid_result.best_params_['max_features'],
                                           n_estimators=grid_result.best_params_['n_estimators'])

# 訓練模型
reg_bestparam.fit(x_train, y_train)

# 預測測試集
y_pred = reg_bestparam.predict(x_test)

# 調整參數後約可降至 0.03 的 MSE
print(metrics.mean_squared_error(y_test, y_pred))

0.04883220046999429
